In [6]:
import os
import glob
from pytorch_grad_cam import GradCAM,AblationCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.transforms import transforms
from PIL import Image

# 클래스 이름 정의 및 모델 로드

In [7]:
folder_to_class = {
    "금속캔알루미늄캔": "metal_can_aluminum",
    "금속캔철캔": "metal_can_steel",
    "비닐": "vinyl",
    "스티로폼": "styrofoam",
    "유리병갈색": "glass_brown",
    "유리병녹색": "glass_green",
    "유리병투명": "glass_clear",
    "종이": "paper",
    "페트병무색단일": "pet_clear",
    "페트병유색단일": "pet_colored",
    "플라스틱PE": "plastic_pe",
    "플라스틱PP": "plastic_pp",
    "플라스틱PS": "plastic_ps"
}
CLASS_NAMES = list(folder_to_class.values())

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(CLASS_NAMES))

model_path = 'model/Best_ResNet50_model.pth'
model.load_state_dict(torch.load(model_path, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

target_layer1 = model.layer4[-1].conv1
target_layer2 = model.layer4[-1].conv2
target_layer3 = model.layer4[-1].conv3

target_layers = [target_layer1, target_layer2, target_layer3]


cam = GradCAM(model=model, target_layers=target_layers)

print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# 전처리

In [8]:
IMG_SIZE = 512
preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 결과 저장 폴더 생성

In [9]:
output_dir = "오분류_GRADCAM_분석/플라스틱PE"
os.makedirs(output_dir, exist_ok=True)

# 이미지 폴더 순회 및 분석

In [10]:
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# 지표가 낮았던 카테고리 폴더 경로를 지정하세요.
input_folder = 'data/Dataset_project4/플라스틱PE' # 분석할 폴더 경로를 변경하세요.

for img_path in glob.glob(os.path.join(input_folder, '*.jpg')):
    true_label_kor = os.path.basename(input_folder)
    true_label_en = folder_to_class.get(true_label_kor)

    rgb_img = Image.open(img_path).convert('RGB')
    rgb_img_np = np.array(rgb_img)
    input_tensor = preprocess(rgb_img).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(input_tensor)
        _, pred_idx = torch.max(output, 1)
        predicted_class_en = CLASS_NAMES[pred_idx.item()]

    # 예측이 틀린 경우에만 GradCAM 진행
    if predicted_class_en != true_label_en:
        # 히트맵 계산
        targets = [ClassifierOutputTarget(pred_idx.item())]
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]
        cam_image = show_cam_on_image(rgb_img_np / 255.0, grayscale_cam, use_rgb=True)

        # 파일명에 실제 라벨과 예측 라벨을 포함시켜 저장
        img_filename = os.path.basename(img_path)
        output_path_original = os.path.join(output_dir, f"원본_{true_label_en}_예측_{predicted_class_en}_{img_filename}")
        output_path_cam = os.path.join(output_dir, f"CAM_{true_label_en}_예측_{predicted_class_en}_{img_filename}")
        
        Image.fromarray(rgb_img_np).save(output_path_original)
        Image.fromarray(cam_image).save(output_path_cam)
        print(f"오분류 이미지와 CAM 저장: {output_path_cam}")

print("분석 완료!")

오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100272@1_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100283@1_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100285@4_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_brown_100302@2_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100305@5_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100315@5_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_brown_100326@1_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_brown_100327@5_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100350@4_04001_220723_P1_T1.jpg
오분류 이미지와 CAM 저장: 오분류_GRADCAM_분석/플라스틱PE\CAM_plastic_pe_예측_glass_green_100373@4_04001_220723_

In [ ]:
import os

# Grad-CAM 결과가 저장된 폴더 경로를 지정합니다.
output_dir = "오분류_GRADCAM_분석"
test_dir = "data/Dataset_project4/종이"

# os.listdir()을 사용하여 폴더 내 모든 파일과 폴더 목록을 가져옵니다.
# isfile() 함수를 사용하여 파일만 필터링합니다.
file_count = len([f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))])

# 파일 개수를 출력합니다.
print(f"'{output_dir}' 폴더에 저장된 오분류 이미지의 총 개수: {file_count}개")

In [ ]:
print(f"'{test_dir}' 폴더에 이미지의 총 개수: {file_count}개")